In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.\
    builder.\
    enableHiveSupport().\
    appName("Spark SQL - Managing Tables - DML and Partitioning").\
    master("yarn").\
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sql("show databases").show()


+------------+
|   namespace|
+------------+
|     default|
|    exercise|
|          hr|
|       kevin|
|kevin_retail|
|      retail|
|         sms|
|        test|
+------------+



In [5]:
spark.sql("use kevin_retail")

DataFrame[]

In [6]:
spark.sql("DROP TABLE IF EXISTS order_items")

DataFrame[]

In [7]:
spark.sql("""CREATE TABLE order_items (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) STORED AS parquet""")

DataFrame[]

In [9]:
spark.sql("DESCRIBE FORMATTED order_items").show(truncate=False)

+----------------------------+------------------------------------------------------------------+-------+
|col_name                    |data_type                                                         |comment|
+----------------------------+------------------------------------------------------------------+-------+
|order_item_id               |int                                                               |null   |
|order_item_order_id         |int                                                               |null   |
|order_item_product_id       |int                                                               |null   |
|order_item_quantity         |int                                                               |null   |
|order_item_subtotal         |float                                                             |null   |
|order_item_product_price    |float                                                             |null   |
|                            |                

 ## LOAD vs. INSERT
 
LOAD will copy the files by dividing them into blocks.
LOAD is the fastest way of getting data into Spark Metastore tables. However, there will be minimal validations at File level. There will be no transformations or validations at data level.

If it require any transformation while getting data into Spark Metastore table, then we need to use INSERT command.

In [10]:
# Loading data from local files and not HDFS
# Above load command will be successful, however when we try to query it will fail as the query expects data
#to be in Parquet file format.
spark.sql("LOAD DATA LOCAL INPATH '/home/hadoop/data/data/retail_db/order_items/' INTO TABLE order_items")

DataFrame[]

In [11]:
spark.sql("SELECT * FROM order_items LIMIT 10").show()

Py4JJavaError: An error occurred while calling o64.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 4) (slave2 executor 2): java.lang.RuntimeException: hdfs://master:9000/user/hive/warehouse/kevin_retail.db/order_items/part-00000 is not a Parquet file. Expected magic number at tail, but found [48, 46, 48, 10]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:556)
	at org.apache.parquet.hadoop.ParquetFileReader.<init>(ParquetFileReader.java:776)
	at org.apache.parquet.hadoop.ParquetFileReader.open(ParquetFileReader.java:657)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFooterReader.readFooter(ParquetFooterReader.java:53)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFooterReader.readFooter(ParquetFooterReader.java:39)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat.footerFileMeta

### Inserting Data using Stage Table

As data is in text file format and our table is created with Parquet file format, we will not be able to use LOAD command to load the data.

In [12]:
spark.sql("TRUNCATE TABLE order_items")

DataFrame[]

In [13]:
spark.sql("SHOW tables").show()

+------------+-----------------+-----------+
|   namespace|        tableName|isTemporary|
+------------+-----------------+-----------+
|kevin_retail|      order_items|      false|
|kevin_retail|order_items_stage|      false|
|kevin_retail|           orders|      false|
|kevin_retail|      orders_part|      false|
+------------+-----------------+-----------+



In [14]:
spark.sql("DROP TABLE IF EXISTS order_items_stage")

DataFrame[]

In [15]:
spark.sql("""CREATE TABLE order_items_stage (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','""")

DataFrame[]

In [16]:
spark.sql("DESCRIBE FORMATTED order_items_stage").show(200, truncate=False)

+----------------------------+------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                               |comment|
+----------------------------+------------------------------------------------------------------------+-------+
|order_item_id               |int                                                                     |null   |
|order_item_order_id         |int                                                                     |null   |
|order_item_product_id       |int                                                                     |null   |
|order_item_quantity         |int                                                                     |null   |
|order_item_subtotal         |float                                                                   |null   |
|order_item_product_price    |float                                                                   |n

In [17]:
spark.sql("LOAD DATA LOCAL INPATH '/home/hadoop/data/data/retail_db/order_items/'  INTO TABLE order_items_stage ")


DataFrame[]

In [18]:
spark.sql("SELECT * FROM order_items_stage LIMIT 10").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [19]:
spark.sql("INSERT INTO TABLE order_items SELECT * FROM order_items_stage limit")

DataFrame[]

In [20]:
spark.sql("SELECT * FROM order_items_stage limit 10").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|        87943|              35210|                 1014|                  4|             199.92|                   49.98|
|        87944|              35211|                  365|                  4|             239.96|                   59.99|
|        87945|              35212|                 1014|                  1|              49.98|                   49.98|
|        87946|              35212|                  191|                  3|             299.97|                   99.99|
|        87947|              35212|                 1014|                  5|              249.9|                   49.98|
|        87948| 

In [21]:
spark.sql("SELECT count(1) FROM order_items").show()

+--------+
|count(1)|
+--------+
|  172198|
+--------+



In [22]:
#INSERT OVERWRITE will overwrite the data in target table by deleting the files related to old data from the 
#directory pointed by the Spark Metastore table.

spark.sql("""INSERT OVERWRITE TABLE order_items
SELECT * FROM order_items_stage limit """)

DataFrame[]

### Creating Partitioned Tables

We can use PARTITIONED BY clause to define the column along with data type.

In [23]:
spark.sql("""CREATE TABLE IF NOT EXISTS orders_part (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) PARTITIONED BY (order_month INT)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','""")

DataFrame[]

In [24]:
spark.sql("DESCRIBE FORMATTED orders_part").show(200, truncate=False)

+----------------------------+------------------------------------------------------------------+-------+
|col_name                    |data_type                                                         |comment|
+----------------------------+------------------------------------------------------------------+-------+
|order_id                    |int                                                               |null   |
|order_date                  |string                                                            |null   |
|order_customer_id           |int                                                               |null   |
|order_status                |string                                                            |null   |
|order_month                 |string                                                            |null   |
|# Partition Information     |                                                                  |       |
|# col_name                  |data_type       

In [25]:
!hdfs dfs -ls /user/hive/warehouse/kevin_retail.db/orders_part/

Found 13 items
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:01 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201307
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:02 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201308
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:02 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201309
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:02 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201310
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:03 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201311
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:05 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201312
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:05 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201401
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 17:05 /user/hive/warehouse/

### Adding Partitions to Tables

We can add partitions using ALTER TABLE command with ADD PARTITION.
For each and every partition created, a subdirectory will be created using partition column name and corresponding value under the table directory.

In [26]:
spark.sql("DROP TABLE IF EXISTS orders_part")

DataFrame[]

In [27]:
spark.sql("""CREATE TABLE IF NOT EXISTS orders_part (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) PARTITIONED BY (order_month STRING)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','""")

DataFrame[]

In [28]:
!hdfs dfs -ls /user/hive/warehouse/kevin_retail.db/orders_part/

In [29]:
spark.sql("DESCRIBE FORMATTED orders_part").show(200, truncate=False)

+----------------------------+------------------------------------------------------------------+-------+
|col_name                    |data_type                                                         |comment|
+----------------------------+------------------------------------------------------------------+-------+
|order_id                    |int                                                               |null   |
|order_date                  |string                                                            |null   |
|order_customer_id           |int                                                               |null   |
|order_status                |string                                                            |null   |
|order_month                 |string                                                            |null   |
|# Partition Information     |                                                                  |       |
|# col_name                  |data_type       

In [30]:
spark.sql("ALTER TABLE orders_part ADD PARTITION (order_month=201307)")

DataFrame[]

In [31]:
spark.sql("""ALTER TABLE orders_part ADD
    PARTITION (order_month=201308)
    PARTITION (order_month=201309)
    PARTITION (order_month=201310)""")

DataFrame[]

In [32]:
!hdfs dfs -ls /user/hive/warehouse/kevin_retail.db/orders_part/

Found 4 items
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201307
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201308
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201309
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201310


### Loading into Partitions

- We need to make sure that file format of the file which is being loaded into table is same as the file format used while creating the table.
- We also need to make sure that delimiters are consistent between files and table for text file format.
- Also data should match the criteria for the partition into which data is loaded.
- Our */home/hadoop/data/data/retail_db/orders* have data for the whole year and hence we should not load the data directly into partition.
- We need to split into files matching partition criteria and then load into the table.

In [33]:
!mkdir -p ~/data/data/partition/orders

In [34]:
!grep 2013-07 /home/hadoop/data/data/retail_db/orders/part-00000 > /home/hadoop/data/data/partition/orders/orders_201307
!grep 2013-08 /home/hadoop/data/data/retail_db/orders/part-00000 > /home/hadoop/data/data/partition/orders/orders_201308
!grep 2013-09 /home/hadoop/data/data/retail_db/orders/part-00000 > /home/hadoop/data/data/partition/orders/orders_201309
!grep 2013-10 /home/hadoop/data/data/retail_db/orders/part-00000 > /home/hadoop/data/data/partition/orders/orders_201310

In [35]:
spark.sql("LOAD DATA LOCAL INPATH '/home/hadoop/data/data/partition/orders/orders_201307'INTO TABLE orders_part PARTITION (order_month=201307)")

DataFrame[]

In [36]:
spark.sql("LOAD DATA LOCAL INPATH '/home/hadoop/data/data/partition/orders/orders_201308' INTO TABLE orders_part PARTITION (order_month=201308)")

DataFrame[]

In [37]:
spark.sql("LOAD DATA LOCAL INPATH '/home/hadoop/data/data/partition/orders/orders_201309' INTO TABLE orders_part PARTITION (order_month=201309)")

DataFrame[]

In [38]:
spark.sql("LOAD DATA LOCAL INPATH '/home/hadoop/data/data/partition/orders/orders_201310' INTO TABLE orders_part PARTITION (order_month=201310)")

DataFrame[]

### Inserting Data into Partitions

In [39]:
spark.sql("ALTER TABLE orders_part ADD PARTITION (order_month=201311)")

DataFrame[]

In [40]:
spark.sql("""INSERT INTO TABLE orders_part PARTITION (order_month=201311)
  SELECT * FROM orders WHERE order_date LIKE '2013-11%'""")

DataFrame[]

In [41]:
spark.sql("SELECT count(1) FROM orders_part").show()

+--------+
|count(1)|
+--------+
|   24770|
+--------+



In [42]:
!hdfs dfs -ls /user/hive/warehouse/kevin_retail.db/orders_part/

Found 5 items
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201307
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201308
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201309
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201310
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201311


## Using Dynamic Partition Mode

- Using dynamic partition mode we need not pre create the partitions. Partitions will be automatically created when we issue INSERT command in dynamic - partition mode.
- To insert data using dynamic partition mode, we need to set the property hive.exec.dynamic.partition to true.
- Also we need to set hive.exec.dynamic.partition.mode to nonstrict.

In [43]:
spark.sql("SET hive.exec.dynamic.partition").show()

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|hive.exec.dynamic...|<undefined>|
+--------------------+-----------+



In [44]:
spark.sql("SET hive.exec.dynamic.partition.mode").show()

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|hive.exec.dynamic...|<undefined>|
+--------------------+-----------+



In [45]:
spark.sql("SET hive.exec.dynamic.partition=true")

DataFrame[key: string, value: string]

In [46]:
spark.sql("SET hive.exec.dynamic.partition.mode=nonstrict")

DataFrame[key: string, value: string]

In [47]:
spark.sql("SET hive.exec.dynamic.partition").show()

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|hive.exec.dynamic...| true|
+--------------------+-----+



In [48]:
spark.sql("SET hive.exec.dynamic.partition.mode").show()

+--------------------+---------+
|                 key|    value|
+--------------------+---------+
|hive.exec.dynamic...|nonstrict|
+--------------------+---------+



In [49]:
spark.sql("""INSERT INTO TABLE orders_part PARTITION (order_month)
SELECT o.*, date_format(order_date, 'yyyyMM') order_month
FROM orders o
WHERE order_date >= '2013-12-01 00:00:00.0'""")

DataFrame[]

In [50]:
!hdfs dfs -ls /user/hive/warehouse/kevin_retail.db/orders_part/

Found 13 items
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201307
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201308
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201309
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201310
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:50 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201311
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:51 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201312
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:51 /user/hive/warehouse/kevin_retail.db/orders_part/order_month=201401
drwxrwxrwx   - hadoop supergroup          0 2023-03-27 19:51 /user/hive/warehouse/

In [54]:
spark.sql("show partitions kevin_retail.orders_part").show()

+------------------+
|         partition|
+------------------+
|order_month=201307|
|order_month=201308|
|order_month=201309|
|order_month=201310|
|order_month=201311|
|order_month=201312|
|order_month=201401|
|order_month=201402|
|order_month=201403|
|order_month=201404|
|order_month=201405|
|order_month=201406|
|order_month=201407|
+------------------+

